In [56]:
### import libraries
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.models import Sequential

from keras.applications.densenet import DenseNet121
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import load_model
from keras.utils.vis_utils import plot_model

import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib
matplotlib.use("Agg") # set the matplotlib backend so figures can be saved in the background
 
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

import datetime
import tensorflow as tf

In [59]:
dtrain=pd.read_csv("CheXpert-v1.0-small/train.csv", nrows=5)
dtrain = dtrain.fillna(0)
# import valid data info from valid.csv
dnew=pd.read_csv("CheXpert-v1.0-small/valid.csv", nrows=5)
dnew = dnew.fillna(0)

#add dnew to dtrain to re-split since valid data in data set is very small
dtrain = dtrain.append(dnew)

#pre-process data: remove Lateral images
dtrain = dtrain[~dtrain[dtrain.columns[3]].str.contains("Lateral")]
#pre-process data: drop selected features - only images as inputs
dtrain = dtrain.drop(["Sex", "Age", "Frontal/Lateral", "AP/PA"], axis=1)
print(dtrain.shape)
dtrain.head()
# dtrain.describe().transpose()

(8, 15)


,Path,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,0.0,0.0,-1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,1.0,0.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,CheXpert-v1.0-small/valid/patient64541/study1/...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# dealing with uncertanty (-1) values
dtrain = dtrain.replace(-1,1)
dtrain.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
No Finding,8141.0,0.097531,0.296698,0.0,0.0,0.0,0.0,1.0
Enlarged Cardiomediastinum,8141.0,0.105638,0.307393,0.0,0.0,0.0,0.0,1.0
Cardiomegaly,8141.0,0.176637,0.381384,0.0,0.0,0.0,0.0,1.0
Lung Opacity,8141.0,0.494902,0.500005,0.0,0.0,0.0,1.0,1.0
Lung Lesion,8141.0,0.040290,0.196650,0.0,0.0,0.0,0.0,1.0
Edema,8141.0,0.310650,0.462788,0.0,0.0,0.0,1.0,1.0
Consolidation,8141.0,0.188920,0.391469,0.0,0.0,0.0,0.0,1.0
Pneumonia,8141.0,0.106498,0.308493,0.0,0.0,0.0,0.0,1.0
Atelectasis,8141.0,0.299840,0.458216,0.0,0.0,0.0,1.0,1.0
Pneumothorax,8141.0,0.104533,0.305969,0.0,0.0,0.0,0.0,1.0


In [49]:
### split data into train/valid/test
dtrain = dtrain.sample(frac=1)
dvalid_size = round(0.1*dtrain.shape[0])
dtest_size = dvalid_size
dtr = dtrain[0:dtrain.shape[0]-dvalid_size-dtest_size+1]
dv = dtrain[dtrain.shape[0]-dvalid_size-dtest_size:dtrain.shape[0]-dvalid_size+1]
dte = dtrain[dtrain.shape[0]-dvalid_size:dtrain.shape[0]+1]

print(dv.shape)

(815, 15)


In [50]:
### data generation for Keras 
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255.)
valid_datagen=ImageDataGenerator(rescale=1./255.)

#target_size = (224,224)
#target_size = (299,299)
target_size = (75,75)
train_generator=train_datagen.flow_from_dataframe(dataframe=dtr, directory=None , x_col="Path", y_col=list(dtr.columns[1:15]), class_mode="other", target_size=target_size, batch_size=32)
valid_generator=valid_datagen.flow_from_dataframe(dataframe=dv, directory=None , x_col="Path", y_col=list(dv.columns[1:15]), class_mode="other", target_size=target_size, batch_size=32)
test_generator=test_datagen.flow_from_dataframe(dataframe=dte, directory=None , x_col="Path", y_col=list(dte.columns[1:15]), class_mode="other", target_size=target_size, shuffle = False, batch_size=1)

Found 6514 validated image filenames.
Found 815 validated image filenames.
Found 814 validated image filenames.


In [51]:
### model architecture design/selection
# create the base pre-trained model
base_model = DenseNet121(include_top = False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(14, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional DenseNet layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)

#model training
adam = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])

#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
#print(model.summary())

In [52]:

### fit model 
num_epochs = 20
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


#Tensorboard
log_dir = "logs/fit/train"# + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_H = model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=num_epochs,
                    callbacks=[tensorboard_callback])
# save model
model.save("model_DenseNet121_TF.h5")

Epoch 1/20
203/203 [==============================] - 151s 584ms/step - loss: 0.5067 - accuracy: 0.1289 - val_loss: 0.4261 - val_accuracy: 0.1200
Epoch 2/20
203/203 [==============================] - 114s 561ms/step - loss: 0.4084 - accuracy: 0.1619 - val_loss: 0.4140 - val_accuracy: 0.2825
Epoch 3/20
203/203 [==============================] - 109s 537ms/step - loss: 0.3985 - accuracy: 0.1772 - val_loss: 0.4110 - val_accuracy: 0.0750
Epoch 4/20
203/203 [==============================] - 96s 475ms/step - loss: 0.3825 - accuracy: 0.1781 - val_loss: 0.4096 - val_accuracy: 0.1275
Epoch 5/20
203/203 [==============================] - 97s 477ms/step - loss: 0.3746 - accuracy: 0.1856 - val_loss: 0.4070 - val_accuracy: 0.1988
Epoch 6/20
203/203 [==============================] - 98s 485ms/step - loss: 0.3713 - accuracy: 0.1899 - val_loss: 0.4070 - val_accuracy: 0.1450
Epoch 7/20
203/203 [==============================] - 99s 487ms/step - loss: 0.3587 - accuracy: 0.2053 - val_loss: 0.4083 - val

In [53]:
# load my trained model 
model_F = load_model('model_DenseNet121_TF.h5')
num_epochs = 3
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [54]:
### prediction and performance assessment
test_generator.reset()
pred=model.predict_generator(test_generator, steps=STEP_SIZE_TEST)
pred_bool = (pred >= 0.5)

y_pred = np.array(pred_bool,dtype =int)

dtest = dte.to_numpy()
y_true = np.array(dtest[:,1:15],dtype=int)

print(classification_report(y_true, y_pred,target_names=list(dtr.columns[1:15])))

/usr/local/lib/python3.9/site-packages/keras/engine/training.py:1976: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


                            precision    recall  f1-score   support

                No Finding       0.55      0.15      0.24        71
Enlarged Cardiomediastinum       0.33      0.01      0.03        77
              Cardiomegaly       0.47      0.14      0.21       152
              Lung Opacity       0.60      0.63      0.62       418
               Lung Lesion       0.00      0.00      0.00        24
                     Edema       0.59      0.51      0.55       263
             Consolidation       0.37      0.10      0.15       165
                 Pneumonia       0.00      0.00      0.00        93
               Atelectasis       0.41      0.10      0.16       265
              Pneumothorax       0.31      0.05      0.08        86
          Pleural Effusion       0.69      0.56      0.62       371
             Pleural Other       0.00      0.00      0.00        26
                  Fracture       0.00      0.00      0.00        35
           Support Devices       0.67      0.49

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  


In [55]:
score, acc = model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)
print('Test score:', score)
print('Test accuracy:', acc)

/usr/local/lib/python3.9/site-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Test score: 0.4298422634601593
Test accuracy: 0.22358722984790802
